In [33]:
import numpy as np
import os
import torch
import importlib
import data_set

importlib.reload(data_set)
#加载图片相关数据
data = data_set.ImageSet()

In [22]:
#加载一些预先训练好的model
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
squeezenet = models.squeezenet1_0(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
densenet = models.densenet161(pretrained=True)
inception = models.inception_v3(pretrained=True)
googlenet = models.googlenet(pretrained=True)
shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
mobilenet = models.mobilenet_v2(pretrained=True)
resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)
mnasnet = models.mnasnet1_0(pretrained=True)


In [34]:
images,lables = data.GetBatch([0,1,2,6])
#直接使用CPU来进行预测运算
with torch.no_grad():
    ret = googlenet(images)
print("lables:")
print(data.category_names[lables])
print("predictions:")
print(data.category_names[ret.topk(1)[1]])

    

lables:
['dung beetle' 'vase' 'bull mastiff' 'strawberry']
predictions:
[['ground beetle, carabid beetle']
 ['pot, flowerpot']
 ['bull mastiff']
 ['strawberry']]


In [35]:
print(data.images[[0,1,2,6]])

['000.png' '001.png' '002.png' '006.png']


In [43]:
a,b = data.__getitem__(0)
print(a.unsqueeze(0).shape)
print(b)
print(torch.tensor([b]).shape)

torch.Size([1, 3, 224, 224])
305
torch.Size([1])
